# In this file we test all NFT functions from nft_analyse.py

## Import

In [ ]:
import sys
# adding signal_handling to the system path
# sys.path.insert(0, '../signal_handling/')
# sys.path.insert(0, '../nft_handling/')
# sys.path.insert(0, '../pjt/')

from FNFTpy import nsev, nsev_poly
from FNFTpy import nsev_inverse, nsev_inverse_xi_wrapper
import numpy as np
import pandas as pd
import random
import timeit
from tqdm import tqdm
from scipy.fft import fft, ifft, fftfreq, fftshift, ifftshift

import matplotlib.pyplot as plt
import matplotlib

from importlib import reload

import ssfm
reload(ssfm)

import signal_generation as sg
reload(sg)

import nft_analyse as nft
reload(nft)

import test_signals
from pjt import pjt

## Create signal

In [ ]:
ampl = 2
chirp = 0.1

t_span = 64.0
n_t = 2 ** 10

dt = t_span / (n_t - 1)
t = np.array([i * dt - t_span / 2. for i in range(n_t)])

xi_span = np.pi / dt
n_xi = 2 ** 8
d_xi = xi_span / (n_xi - 1)
xi = np.array([i * d_xi - xi_span / 2. for i in range(n_xi)])

q, a_xi, b_xi, xi_discr, b_discr, r_discr, ad_discr = test_signals.get_sech(t, xi, a=ampl, c=chirp)
print(len(xi_discr), xi_discr)
z_prop = 4.3
q_prop = ssfm.fiber_propogate(q, t_span, z_prop, n_span=2 ** 9, gamma=1, beta2=-1)

matplotlib.rcParams.update({'font.size': 30})
# color = ['red', 'blue', 'green', 'xkcd:light purple', 'xkcd:cyan']
fig, axs = plt.subplots(1, 1, figsize=(20, 5))
axs.plot(t, np.absolute(q), 'xkcd:cyan', linewidth=3)
axs.plot(t, np.absolute(q_prop), 'xkcd:lime green', linewidth=3)
axs.grid(True)

# print(b_xi)

result_cont = get_continuous_spectrum(q, t, xi=None, type='fnft', xi_upsampling=1, fnft_type=0, nft_type='bo', res_poly=None, coefficient_type='left')

In [ ]:
z_back = z_prop
xi_upsampling = 1
forward_continuous_type='fnft'
forward_discrete_type='fnft'
forward_discrete_coef_type='bi-direct'
inverse_type='both'
fnft_type=0
nft_type='bo'
use_contour = False
n_discrete_skip = 2

res_dbp = nft.make_dbp_nft(q_prop, t, z_back, xi_upsampling=xi_upsampling,
                           forward_continuous_type=forward_continuous_type,
                           forward_discrete_type=forward_discrete_type,
                           forward_discrete_coef_type=forward_discrete_coef_type,
                           inverse_type=inverse_type,
                           fnft_type=fnft_type, nft_type=nft_type,
                           use_contour=use_contour, n_discrete_skip=n_discrete_skip,
                           print_sys_message=True)

q_tib_total = res_dbp['q_total']
q_tib_left = res_dbp['q_tib_left']
q_tib_right = res_dbp['q_tib_right']
q_fnft = res_dbp['q_fnft']

xi_d_calculated = res_dbp['xi_d']
print(xi_d_calculated)
print(xi_discr)




# for fnft len(xi) == len(q)
res = nsev_inverse(xi, t, b_xi, xi_discr, b_discr / ad_discr, cst=1, dst=0, dis=fnft_type)
q_fnft_still = res['q']

print(np.shape(b_xi), np.shape(xi))

In [ ]:
matplotlib.rcParams.update({'font.size': 30})
color = ['red', 'blue', 'green', 'xkcd:light purple', 'xkcd:cyan', 'xkcd:lime green']
fig, axs = plt.subplots(2, 1, figsize=(20, 10))
axs[0].plot(t, np.absolute(q), color[0], linewidth=10)
axs[0].plot(t, np.absolute(q_tib_total), color[1], linewidth=7)
axs[0].plot(t, np.absolute(q_tib_left), color[2], linewidth=5)
axs[0].plot(t, np.absolute(q_tib_right), color[3], linewidth=3)
axs[0].plot(t, np.absolute(q_fnft), color[4], linewidth=3)
axs[0].plot(t, np.absolute(q_fnft_still), color[5], linewidth=3)
axs[0].grid(True)
axs[0].set_xlim(-5, 5)
axs[0].set_ylim(0, ampl + 0.5)

axs[1].plot(t, np.absolute(q - q_tib_total), color[1], linewidth=10)
axs[1].plot(t, np.absolute(q - q_tib_left), color[2], linewidth=3)
axs[1].plot(t, np.absolute(q - q_tib_right), color[3], linewidth=3)
axs[1].plot(t, np.absolute(q - q_fnft), color[4], linewidth=3)
axs[1].plot(t, np.absolute(q - q_fnft_still), color[5], linewidth=3)
axs[1].grid(True)
axs[1].set_yscale('log')

#set y ticks
y_major = matplotlib.ticker.LogLocator(base = 10.0, numticks = 5)
axs[1].yaxis.set_major_locator(y_major)
y_minor = matplotlib.ticker.LogLocator(base = 10.0, subs = np.arange(1.0, 10.0) * 0.1, numticks = 10)
axs[1].yaxis.set_minor_locator(y_minor)
axs[1].yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())